# Analyse des données textuelles

1. [Les descriptions](#descr)
2. [Les Titres](#Les Titres)

On souhaite :
    * titres
    * Description

https://openclassrooms.com/fr/courses/4470541-analysez-vos-donnees-textuelles/

Le texte que vous utilisez, que l'on appelle « corpus »,

<a id = "division_ID"text goes here></a>
## récupération du texte

Mes dictionnaires sont constitués d'objets de la forme :
* { title, video_id }
* {description,video_id}

In [5]:
import pandas as pd

In [7]:
fr_videos = pd.read_csv("../data/modif_FRvideos.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
fr_videos.shape

(40854, 19)

<a id = "descr"Les descriptions></a>

## Les descriptions

In [ ]:
On regarde si il y a des descriptions vides (NaN)

In [9]:
fr_videos[fr_videos["description"].apply(lambda x: pd.isna(x))].head(3)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category,engagement,duree
63,SoDxxEyxE84,17.14.11,Raquel Garrido explique son départ de FI - Les...,Gaetano Gitano,22.0,2017-11-12T21:30:42.000Z,[none],12740.0,110.0,16.0,69.0,https://i.ytimg.com/vi/SoDxxEyxE84/default.jpg,False,False,False,NaN,People & Blogs,0.014050,NaN
75,tlSLFSw_lYs,17.14.11,Kabila pousse Tshibala vers la porte de sortie...,lokoleelela,25.0,2017-11-13T08:35:59.000Z,[none],26260.0,182.0,44.0,156.0,https://i.ytimg.com/vi/tlSLFSw_lYs/default.jpg,False,False,False,NaN,News & Politics,0.012871,NaN
94,rfZuiD2MaUM,17.14.11,Karl Lagerfeld descend Merkel dans Salut les T...,SQO,22.0,2017-11-11T19:09:59.000Z,[none],23722.0,88.0,25.0,60.0,https://i.ytimg.com/vi/rfZuiD2MaUM/default.jpg,False,False,False,NaN,People & Blogs,0.006239,NaN


Observation : Il y a des NaN. Il faut les traiter.
    Soit on remplace les NaN par des string vide
    soit on supprime les lignes concernées

### Remplacement des NaN par des string vide

In [10]:
fr_videos["description"] = fr_videos["description"].fillna(value="")

In [11]:
fr_videos[fr_videos["description"].apply(lambda x: pd.isna(x))].head(3)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category,engagement,duree


Plus de NaN

## Les Titres

### Créer un dictionnaire { title : video_id }

In [18]:
fr_dico_titres = dict(zip(fr_videos['title'], fr_videos['video_id']))

In [19]:
type(fr_dico_titres)

dict

In [27]:
len(fr_dico_titres.keys())

30560

In [20]:
print('chargement de {} titres dans la db'.format(len(fr_dico_titres.keys())))

chargement de 30560 titres dans la db


### Suppimer les lignes sans titre du DF

manipulation pour comparer le résultat avec la taille du dictionnaire

est-ce que toutes les vidéos ont un titre ?

In [23]:
fr_videos[fr_videos["title"].apply(lambda x: pd.isna(x))].head(3)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category,engagement,duree
33837,Retrouvez « le Journal de 17h17 » dans Par Jup...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
35780,Subscribe to WWE on YouTube: http://bit.ly/1i6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
35781,Visit WWE.com: http://goo.gl/akf0J4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN


Suppression des lignes sans titre

In [24]:
fr_videos["title"] = fr_videos["description"].dropna(how='all')

In [25]:
fr_videos[fr_videos["title"].apply(lambda x: pd.isna(x))].head(3)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category,engagement,duree


In [29]:
fr_videos["title"].shape

(40854,)

### Classement des mots les plus utilisés dans les titres

fr_dico_titres

In [16]:
title_words = list(fr_videos["title"].apply(lambda x: x.split()))
title_words = [x for y in title_words for x in y]
Counter(title_words).most_common(25)

AttributeError: 'float' object has no attribute 'split'

## Exploration du texte : tokenisation et analyse des fréquences

On veut le vocabulaire utilisé pour chaque videos

nombre de mots utilisés
fonction word_tokenize
décomposer les vers en tableaux de mots

### test

In [30]:
import nltk

lancer l'interface graphique de nltk

In [31]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Installer le module punkt via l'interface

In [4]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
test = "Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !"

nltk.word_tokenize(test)

['Bonjour',
 ',',
 'je',
 'suis',
 'un',
 'texte',
 "d'exemple",
 'pour',
 'le',
 'cours',
 "d'Openclassrooms",
 '.',
 'Soyez',
 'attentifs',
 'à',
 'ce',
 'cours',
 '!']

Nettoyage
* la ponctuation
* apostrophes : "d'exemple" devrait être séparé en "de" et "exemple"
* majuscules



normalisation

harmoniser les tokens

récupérer que les caractères alphanumériques de chaque phrase  avec les expressions régulières

https://www.debuggex.com/cheatsheet/regex/python

In [33]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokenizer.tokenize("Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !")


['Bonjour',
 'je',
 'suis',
 'un',
 'texte',
 'd',
 'exemple',
 'pour',
 'le',
 'cours',
 'd',
 'Openclassrooms',
 'Soyez',
 'attentifs',
 'à',
 'ce',
 'cours']

normalisation des données

In [45]:
from collections import defaultdict
tokenizer = nltk.RegexpTokenizer(r'\w+')

liste = defaultdict(list)

# Création d'un corpus de tokens par videos
for video_id in fr_dico_titres.items():
        liste[video_id] += tokenizer.tokenize(fr_dico_titres[video_id]['title'])


SyntaxError: invalid syntax (<ipython-input-45-a7e25131d6f9>, line 8)

In [ ]:
dico_tags_sorted = sorted(dico_tags.items(),key=lambda c:c[1], reverse=True)
dico_tags_sorted

In [ ]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(preserve_case=False)
tokens = tknzr.tokenize(data.loc[0, "text"])
tokens

In [49]:
from collections import defaultdict
tokenizer = nltk.RegexpTokenizer(r'\w+')

def freq_stats_corpora():
    corpora = defaultdict(list)

    # Création d'un corpus de tokens par videos
    for video,sentence_id in fr_dico_titres.items():
        for sentence_id in sentence_id:
            #corpora[video] += tokenizer.tokenize(fr_dico_titres[sentence_id]['title'].decode('utf-8').lower())
            corpora[video] += tokenizer.tokenize(fr_dico_titres[sentence_id]['title'])

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v)} 
    return (freq, stats, corpora)

# Récupération des comptages
freq, stats, corpora = freq_stats_corpora()
fr_dico_titres = fr_dico_titres.DataFrame.from_dict(stats, orient='index')

# Affichage des fréquences
fr_dico_titres.sort(columns='total', ascending=False)
fr_dico_titres.plot(kind='bar', color="#f56900", title='Top 50 vidéos par nombre de mots')

TypeError: first argument must be callable or None

In [ ]:
fr_dico_titresbyid = dict(zip(fr_videos['video_id'], fr_videos['title']))

In [44]:
fr_dico_titresbyid

{'Ro6eob0LrCY': 'Malika LePen : Femme de Gauche - Trailer',
 'Yo84eqYwP98': "LA PIRE PARTIE ft Le Rire Jaune, Pierre Croce, Fabien Olicard, Nad Rich' Hard, Max Bird, Studio Vrac",
 'ceqntSXE-10': 'DESSINS ANIMÉS FRANÇAIS VS RUSSES 2 - Daniil le Russe',
 'WuTFI5qftCE': 'PAPY GRENIER - METAL GEAR SOLID',
 'ee6OFs8TdEg': 'QUI SAUTERA LE PLUS HAUT ? (VÉLO SKATE ROLLER TROTTINETTE)',
 'teXaL6GdQRk': 'STRANGER JOKES : Jokes de Papa avec les teens de Stranger Things',
 'nduL7G_gJoY': 'De retour dans le Manoir hanté avec le Grand JD !!',
 'QmpWE_SODZA': "T'es qui toi ? Squeezie, le youtubeur aux 4 milliards de vues - Salut les Terriens",
 'GBVxEpQr8R8': 'ON VOUS DÉVOILE NOTRE VRAI SALAIRE',
 '0RFhWyM6qbA': 'Benzema balance ses dur vérités Deschamps et Les bleus Dans le CFC !',
 'LhjhGOBIoHM': "Jérémy Ferrari - On n'est pas couché 11 novembre 2017 #ONPC",
 'lnFaRuWOLN0': 'Emilie, 10 ans, Coach De Vie',
 'LGbUBietbJc': "J'ABANDONNE UNE PETITE FILLE DEVANT DES INCONNUS (Social Experiment)",
 'J

In [50]:
fr_dico_titresbyid = tokenizer.tokenize(fr_dico_titresbyid.decode('utf-8').lower())

AttributeError: 'dict' object has no attribute 'decode'

In [47]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
for key in fr_dico_titresbyid:
    list = tokenizer.tokenize(fr_dico_titresbyid.values())

TypeError: expected string or bytes-like object

In [46]:
print(list)

['Mandoumbé', 'ak', 'Koor', 'Gui', '2018', 'Episode', '28']


https://code.tutsplus.com/fr/tutorials/introducing-the-natural-language-toolkit-nltk--cms-28620